In [1]:
from pyspark.sql import SparkSession
from delta.tables import *
from delta import *

In [2]:
 spark = SparkSession.builder \
        .appName("ProcessJSONToDelta") \
        .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.3.0") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()

25/02/04 09:04:59 WARN Utils: Your hostname, NB-WXET1590 resolves to a loopback address: 127.0.1.1; using 172.30.88.241 instead (on interface eth0)
25/02/04 09:04:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/avyas/miniconda3/envs/testairbyte/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/avyas/.ivy2/cache
The jars for the packages stored in: /home/avyas/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e4dd0187-b964-425f-9df8-4cf2fa321ba7;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 159ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

In [74]:
delta_table = DeltaTable.forPath(spark, '../data/delta_table/core_data/')

In [75]:
schema = delta_table.toDF().schema
schema

StructType([StructField('abstract', StringType(), True), StructField('authors', ArrayType(StringType(), True), True), StructField('contributors', ArrayType(StringType(), True), True), StructField('coreId', StringType(), True), StructField('datePublished', StringType(), True), StructField('doi', StringType(), True), StructField('downloadUrl', StringType(), True), StructField('enrichments', StructType([StructField('documentType', StructType([StructField('type', DoubleType(), True)]), True), StructField('references', ArrayType(StructType([StructField('authors', ArrayType(StringType(), True), True), StructField('cites', StringType(), True), StructField('date', StringType(), True), StructField('doi', StringType(), True), StructField('id', LongType(), True), StructField('raw', StringType(), True), StructField('title', StringType(), True)]), True), True)]), True), StructField('fullText', StringType(), True), StructField('fullTextIdentifier', StringType(), True), StructField('identifiers', Arr

In [76]:
metadata = delta_table.history()
metadata.show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      1|2025-02-01 05:10:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|          0|  Serializable|        false|{numFiles -> 683,...|        NULL|Apache-Spark/3.5....|
|      0|2025-02-01 05:09:...|  NULL|    NULL|    WRITE|{mode -> Append, ...|NULL|    NULL|     NULL|       NULL|  Serializable|         true|{numFiles -> 683,...|        NULL|Apache-Spark/3.5....|
+-------+-

In [77]:
delta_table.toDF().show(1)

+--------------------+--------------------+------------+------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+---------+--------------------+---------+--------------------+--------------------+------+----+
|            abstract|             authors|contributors|coreId|datePublished|                 doi|         downloadUrl|enrichments|            fullText|  fullTextIdentifier|         identifiers|            journals|        language|                 oai|        pdfHashValue|publisher|        rawRecordXml|relations|             subject|               title|topics|year|
+--------------------+--------------------+------------+------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+-------

In [78]:
spark.sql ("""CREATE TABLE delta_table
                USING delta
                LOCATION '/home/avyas/projects/project_os/elt_articles//data/delta_table/core_data/'""")

AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `default`.`delta_table` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

In [57]:
# spark.sql("DROP TABLE delta_table1")

++
||
++
++



In [79]:
spark.sql("SHOW TABLES").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|  default|delta_table|      false|
+---------+-----------+-----------+



In [68]:
spark.sql("SELECT current_database()").show()

+------------------+
|current_database()|
+------------------+
|           default|
+------------------+



In [80]:
spark.sql("""DESCRIBE delta_table""").show()

+------------------+--------------------+-------+
|          col_name|           data_type|comment|
+------------------+--------------------+-------+
|          abstract|              string|   NULL|
|           authors|       array<string>|   NULL|
|      contributors|       array<string>|   NULL|
|            coreId|              string|   NULL|
|     datePublished|              string|   NULL|
|               doi|              string|   NULL|
|       downloadUrl|              string|   NULL|
|       enrichments|struct<documentTy...|   NULL|
|          fullText|              string|   NULL|
|fullTextIdentifier|              string|   NULL|
|       identifiers|       array<string>|   NULL|
|          journals|array<struct<iden...|   NULL|
|          language|struct<code:strin...|   NULL|
|               oai|              string|   NULL|
|      pdfHashValue|              string|   NULL|
|         publisher|              string|   NULL|
|      rawRecordXml|              string|   NULL|


In [99]:
spark.sql("""Select COUNT(*) from delta_table """).show()

+--------+
|count(1)|
+--------+
|   21840|
+--------+



In [100]:
spark.sql("""SELECT doi, COUNT(*) as count
            FROM delta_table
            GROUP BY doi
            HAVING COUNT(*) > 1;
            """).show()

+--------------------+-----+
|                 doi|count|
+--------------------+-----+
|10.1017/S01433857...|    2|
|        10.1093/iclq|   14|
|      10.1093/indlaw|   13|
|         10.1093/jeg|   27|
|      10.1093/restud|    2|
|        10.1093/jcsl|    2|
|                NULL| 1147|
|   10.1088/0951-7715|    5|
|   10.1088/0960-1317|    4|
|10.1137/S00361445...|    2|
|10.1186/1745-6215...|    2|
|        10.1155/2008|    6|
|       10.1093/cjres|    2|
|10.5194/angeo-21-...|    2|
|        10.1093/past|    6|
|   10.1088/1126-6708|   20|
|      10.1093/jopart|    4|
|10.1525/srsp.2008...|    2|
|   10.1137/060657984|    2|
|10.1017/S01433857...|    2|
+--------------------+-----+
only showing top 20 rows



25/02/04 13:09:10 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1219)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:295)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java